---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 2 - Pandas Introduction
All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [31]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*

In [2]:
# You should write your whole answer within the function provided. The autograder will call
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting
    return df.iloc[0]

# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
answer_zero() 

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [3]:
def answer_one():
    a=df['Combined total'].max()
    max_index=df[df['Combined total']==a]
    return max_index.index[0]

'United States'

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [4]:
def answer_two():
    df['difference_sw']=df['Gold']-df['Gold.1']
    a=df['difference_sw'].max()
    return df[df['difference_sw']==a].index[0]

'Egypt'

### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [8]:
def answer_three():
    df['re_difference_sw']=(df['Gold']-df['Gold.1'])/(df['Gold.2'])
    df_2=df[(df['Gold'] > 0) & (df['Gold.1'] > 0)]
    a=df_2['re_difference_sw'].max()
    return df_2[df_2['re_difference_sw']==a].index[0]

'Bulgaria'

### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created, with the country names as indices.

*This function should return a Series named `Points` of length 146*

In [15]:
def answer_four():
    df['Points']=df['Gold.2']*3+df['Silver.2']*2+df['Bronze.2']
    return df['Points']

Afghanistan                            2
Algeria                               27
Argentina                            130
Armenia                               16
Australasia                           22
Australia                            923
Austria                              569
Azerbaijan                            43
Bahamas                               24
Bahrain                                1
Barbados                               1
Belarus                              154
Belgium                              276
Bermuda                                1
Bohemia                                5
Botswana                               2
Brazil                               184
British West Indies                    2
Bulgaria                             411
Burundi                                3
Cameroon                              12
Canada                               846
Chile                                 24
China                               1120
Colombia        

## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2015/co-est2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [34]:
census_df = pd.read_csv('census.csv')
census_df.head()

,Unnamed: 0,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,1,50,3,6,1,1,Alabama,Autauga County,54571,54571,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,3,50,3,6,1,5,Alabama,Barbour County,27457,27457,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,4,50,3,6,1,7,Alabama,Bibb County,22915,22919,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In [37]:
def answer_five():
    number=0
    max1=0
    to_compare='Alabama' 
    for i in census_df['STNAME']:
        if i==to_compare:
            number+=1
        else:
            if number>max1:
                max1=number
                state=to_compare
            number=1
        to_compare=i
        if number>max1:
            state=i
    return state


### Question 6
**Only looking at the three most populous counties for each state**, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

In [33]:
def answer_six():
    census_df2=census_df.set_index(['STNAME'])
    state1_count=0
    state2_count=0
    state3_count=0
    state1=''
    state2=''
    state3=''
    for i in census_df2.index.unique():
        a=sum(census_df2.loc[i,'CENSUS2010POP'])
        if state1_count<a:
            state1_count=a
            state1=i
        else:
            if state2_count<a:
                state2_count=a
                state2=i
            else:
                if state3_count<a:
                    state3_count=a
                    state3=i
    return [state1,state2, state3]

### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [38]:
def answer_seven():
    biggest_dif=0
    for i in census_df.index:
        max1=max([census_df.loc[i,'POPESTIMATE2010'],census_df.loc[i,'POPESTIMATE2011'],census_df.loc[i,'POPESTIMATE2012'],census_df.loc[i,'POPESTIMATE2013'],census_df.loc[i,'POPESTIMATE2014'],census_df.loc[i,'POPESTIMATE2015']])
        min1=min([census_df.loc[i,'POPESTIMATE2010'],census_df.loc[i,'POPESTIMATE2011'],census_df.loc[i,'POPESTIMATE2012'],census_df.loc[i,'POPESTIMATE2013'],census_df.loc[i,'POPESTIMATE2014'],census_df.loc[i,'POPESTIMATE2015']])
        print(min1,max1)
        deff=max1-min1
        if deff>biggest_dif:
            biggest_dif=deff
            state=census_df.loc[i,'STNAME']
    return state 
answer_seven()

4785161 4858979
54660 55347
183193 203709
26489 27341
22512 22861
57373 57776
10606 10887
20154 20944
115620 118437
33993 34153
25859 26084
43665 43943
13170 13841
24675 25767
13456 13880
14921 15072
50177 51211
54354 54514
12662 13208
10724 11758
37796 38060
13853 13963
80374 82005
49501 50358
41131 43803
70869 71387
79465 81468
37784 38309
103057 104442
16759 17231
31507 31734
26707 27006
8479 9001
15068 15735
17176 17407
101783 104173
52419 53264
657704 660368
13886 14495
92596 93063
33115 34305
140782 156993
83163 91663
10458 11275
19105 21545
336138 353089
20028 20942
30168 30800
93146 94725
413062 415395
21673 22982
226389 231496
119565 120112
9652 10540
19355 20864
32903 33630
22486 22891
53297 59712
83812 87074
196048 208713
13103 13740
80862 82071
40844 41502
194977 203976
65294 67004
16804 17610
11052 11557
23877 24401
714021 738432
3169 3341
5546 5751
293405 301537
17080 17946
892 1038
1837 1934
4856 4997
98146 101007
2501 2570
2091 2147
31388 32756
55573 58059
13525 13781
1

21543 22328
16387 16723
15989 16386
4226 4490
5678 6134
5644 5999
32852 33458
16029 16231
146133 147713
264052 270364
24548 24969
197797 199296
7032 7542
5092 5335
21775 22359
5788 5970
45749 47656
134800 136242
17408 17746
79282 81613
11542 11905
17527 17844
14270 14701
16423 16746
14327 14611
57079 58496
678910 687263
66440 67466
287078 295101
38655 39234
6490590 6619680
34374 34980
355890 368450
76846 81162
8681 8856
12298 12750
56845 63344
14932 15200
19856 20183
37979 38974
110557 115371
26483 26881
32609 33203
10483 10716
31744 32906
49423 50099
25791 26521
42293 42589
116852 117824
41906 42461
197471 203474
23434 24333
74708 76778
16591 17271
22872 23062
20315 20818
33554 33845
67979 69935
32441 33166
276501 309697
70223 72520
39051 39578
145863 158192
48985 49488
82556 82900
36630 37187
42578 44069
33412 33514
21121 21370
32302 32529
27897 28475
140281 149633
37845 38467
77340 78620
37156 38809
487865 495987
110884 111695
45495 46124
129723 131637
904668 939020
46857 47013
1020

6856 7237
10631 10873
3917 4032
27650 27809
5771 5885
25593 25859
35891 36605
5424 5525
9423 9480
20022 20818
23070 23321
25737 26095
32775 33240
39116 39312
8413 8692
32747 33347
21398 21770
6640 6849
144522 151436
57282 57716
13961 14219
29069 29723
9271 9593
31489 31658
10912 11041
509428 538133
4019 4103
15471 16066
14892 15687
64243 65400
9516 9664
15507 15770
200158 200780
130537 141660
88799 91705
14875 15234
150749 154708
36302 36755
9715 9838
9340 9753
24257 24874
3391 3541
21239 21663
13697 13875
18965 19184
238995 251597
10952 11197
6396 6614
50885 51419
125137 131311
9875 10425
2970316 2993443
31254 32560
37108 37388
12574 13176
19043 19540
8182 8705
33322 34091
14717 14952
10243 10580
17328 17469
8299 8561
9150 9780
16006 16707
20048 20547
24620 26121
28773 29428
19378 19595
161788 173323
74998 76766
7743 8117
22659 23373
13522 14404
21540 21855
44102 46420
187896 201410
242891 248324
18340 19400
8669 9330
1337 1413
23333 23609
139453 141504
16481 16988
7507 7711
11665 124

104806 107863
36568 36928
42306 43769
1255921 1278226
52076 52962
38352 39112
175146 193013
75550 77034
146385 151408
28679 29017
1166296 1251722
42355 42616
30142 31085
93374 94102
161608 164660
39258 40136
800642 807598
74680 75653
31638 32099
15450 15849
27816 28223
43023 43598
28491 29475
42470 43909
58469 59578
32596 33273
67347 69612
60805 61307
229220 230004
61109 62420
166719 170570
45386 45770
301471 305147
433689 441575
42949 44094
231900 238398
65355 66543
172542 176395
23257 23733
40772 40968
102479 104224
14409 14719
532258 536216
14777 15088
34815 35074
85742 86290
14326 14700
40877 41414
18976 19575
35910 36233
55745 56998
28217 28738
161448 162275
41329 42171
34042 34459
121707 124173
77115 78101
59679 60874
76825 79528
55610 56630
48901 49313
374292 375638
541192 542600
203751 209854
92501 92916
52420 54277
28442 28718
13048 13421
213524 224469
61112 61708
114439 116063
37120 37586
125940 129730
22243 22659
3759596 3911338
22004 22731
5632 5868
13793 14161
5427 5638
22

130859 132955
12944 13502
21903 22201
426488 508514
43075 47520
7080 8005
59108 62953
42007 43356
7845 8546
18270 18543
14075 14374
11711 12235
2775426 2995919
6354 6653
50160 52097
113367 120783
20479 21421
1074 1166
307929 336043
18631 20862
10370 10992
5009 5185
9278 9516
46267 48368
10259 10594
7131 7235
12514 12645
9528 11065
1486 1556
2251 2313
1033182 1107314
14796 15772
27879 28778
20753 20984
36512 39633
58517 62952
32444 37928
520011 575205
23639 29161
138424 155602
2692 2765
232183 243645
625984 627129
36820 37035
36317 37105
30780 31181
156805 161382
6129 6328
47797 48799
6861 6979
24537 25235
28868 29053
27086 27246
59736 61585
58612 59570
43386 44509
55737 56649
8025787 8382993
32973 33314
99236 105703
15677 16340
12698 12903
31914 32449
15003 15414
209429 229164
73478 74314
4470 4712
74997 77724
6561 6810
33053 33347
16698 17413
22776 24040
16927 17181
54888 55101
28643 29984
29641 30096
7024 7271
12186 12547
317312 335687
14038 14365
5176 5240
46848 49432
9719 10044
151

'Texas'

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [30]:
def answer_eight():
    STNAME=[]
    CTYNAME=[]
    for i in census_df.index:
        if census_df.loc[i,'REGION']==1 or census_df.loc[i,'REGION']==2:
            if len(census_df.loc[i,'CTYNAME'])>=10:
                if 'Washington'== census_df.loc[i,'CTYNAME'][0:10]:
                    if census_df.loc[i,'POPESTIMATE2015']>census_df.loc[i,'POPESTIMATE2014']:
                        STNAME.append(census_df.loc[i,'STNAME'])
                        CTYNAME.append(census_df.loc[i,'CTYNAME'])
    return pd.DataFrame({'STNAME':STNAME,'CTYNAME':CTYNAME})

,CTYNAME,STNAME
0,Washington County,Iowa
1,Washington County,Minnesota
2,Washington County,Pennsylvania
3,Washington County,Rhode Island
4,Washington County,Wisconsin
